## 2.2 붓꽃 품종 예측하기 (decision tree)

- Classification은 대표적인 지도학습 방법

* 지도학습 : 학습을 위한 다양한 피처와 분류 결정값인 label 데이터로 모델 학습 -> 테스트 데이터로 미지의 label예측 >> 명확한 정답이 주어진 데이터를 먼저 학습한 뒤 정답 예측하는 방법

* 비지도학습 : 차원 축소, 클러스터링, 피처 추출

In [1]:
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
import pandas as pd

In [2]:
iris = load_iris()
iris_data = iris.data

In [3]:
iris_label=iris.target
iris.target_names

array(['setosa', 'versicolor', 'virginica'], dtype='<U10')

In [4]:
iris_df=pd.DataFrame(data = iris_data, columns = iris.feature_names)
iris_df


,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2
...,...,...,...,...
145,6.7,3.0,5.2,2.3
146,6.3,2.5,5.0,1.9
147,6.5,3.0,5.2,2.0
148,6.2,3.4,5.4,2.3


In [5]:
iris_df['label']=iris.target # <- label = 결정값 , setosa :1 , versicolor : 2, virginica : 3

In [6]:
#train, test data split : train_test_split() 사용
X_train, X_test, y_train, y_test = train_test_split(iris_data, iris_label, test_size =0.2, random_state =11)
#iris_data = feature data
#iris_label = label data set
#random_stae를 설정하지 않으면 함수를 호출할 때마다 다른 결과가 나온다.(in R, set.seed와 같음)

In [7]:
#decisoin tree 방법으로 label 예측
tree = DecisionTreeClassifier(random_state =11)
tree.fit(X_train,y_train) # <---학습 완료

DecisionTreeClassifier(random_state=11)

In [8]:
# 예측하기
import numpy as np
pred = tree.predict(X_test)

In [9]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,pred) # <- accuracy

0.9333333333333333

## 2.3 사이킷런의 기반 프레이워크 익히기

### Estimator 이해 및 fit(), predict() 메서드

* 지도학습 : fit() & predict()
* 비지도학습 : fit() & transform()

target = 예측치라고 생각하기

## 2.4 Model Selection 모듈 소개

In [10]:
# train/test data split --> train_test_split()

from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

In [11]:
iris= load_iris()
model_tree = DecisionTreeClassifier()
train = iris.data
train_label = iris.target
model_tree.fit(train,train_label)

DecisionTreeClassifier()

In [12]:
pred = model_tree.predict(train)
accuracy_score(train_label,pred) # accuracy_score(test data label , train model predict)

# accuracy가 1이 나온 이유 : train_label로 예측했기 때문

1.0

train, test로 나눠서 실시 (data는 data와 target로 나눠짐. target : 실제 예측 값)

In [13]:
model_tree = DecisionTreeClassifier()
iris = load_iris()

X_train , X_test , y_train, y_test = train_test_split(iris.data , iris.target, test_size = 0.3, random_state = 121)

In [14]:
model_tree.fit(X_train,y_train)
pred = model_tree.predict(X_test)

accuracy_score(y_test, pred)

0.9555555555555556

총 데이터의 크기가 크지 않아 모델의 성능을 판단할 수 없다. 95.6%가 나왔어도!

2) 교차검증

In [15]:
from sklearn.model_selection import KFold
import numpy as np

iris = load_iris()
features = iris.data
label = iris.target
model_tree = DecisionTreeClassifier(random_state = 156)

#K=5로 Kfold 실시
kfold = KFold(n_splits = 5)
cv_accuracy = []
print('붓꽃 데이터 세트의 크기 : ', features.shape[0])


붓꽃 데이터 세트의 크기 :  150


In [16]:
n_iter = 0

# fold 별 accuraccy 확인

for train_index, test_index in kfold.split(features) : 
    X_train, X_test = features[train_index], features[test_index]
    y_train, y_test = label[train_index], label[test_index]
    
    #model fit & predict
    model_tree.fit(X_train, y_train)
    pred = model_tree.predict(X_test)
    n_iter += 1
    
    #정확도 측정
    accuracy = np.round(accuracy_score(y_test, pred),4)
    cv_accuracy.append(accuracy)
    
print(cv_accuracy)
np.mean(cv_accuracy)


[1.0, 0.9667, 0.8667, 0.9333, 0.7333]


0.9

Stratified K fold method
- imbalanced label distribution일 때 사용 (imbalanced : 특정 레이블 값이 특이하게 많거나 적은 경우. 분포가 치우칠 때)

ex) 대출 사기 데이터

- 원본 데이터와 비슷한 분포로 k fold 데이터를 생성하는 것이 중요
- stratified K fold는 원본 데이터의 분포를 먼저 살펴 봐야한다.

In [17]:
iris = load_iris()
iris_df = pd.DataFrame(iris.data , columns = iris.feature_names)

iris_df['label'] = iris.target
iris_df['label'].value_counts()

2    50
1    50
0    50
Name: label, dtype: int64

In [20]:
# cross validation <- stratified K fold 방식으로 분할
from sklearn.model_selection import cross_val_score, cross_validate

iris=load_iris()
model_tree = DecisionTreeClassifier(random_state = 156)

data = iris.data
label = iris.target

#성능 지표는 accuracy, k = 3
scores = cross_val_score(model_tree, data, label, scoring = 'accuracy', cv= 3)
print('cv accuracy : ', np.round(scores,4))
print('mean of cv accuracy : ' , np.mean(scores))

cv accuracy :  [0.98 0.94 0.98]
mean of cv accuracy :  0.9666666666666667


- GridSearchCV : 교차 검증과 최적 하이퍼 파라미터 튜닝을 한 번에

In [24]:
from sklearn.model_selection import GridSearchCV

X_train, X_test , y_train, y_test = train_test_split(iris.data, iris.target, test_size = 0.2, random_state = 121)
model_tree = DecisionTreeClassifier()

## 파라미터를 딕셔너리 형태로 설정
param = {'max_depth' : [1,2,3], 'min_samples_split':[2,3]}

In [25]:
# param_grid의 하이퍼 파라미터를 3개의 train, test set fold로 나누어 테스트 수행 결정
grid_tree =GridSearchCV(model_tree,param_grid = param, cv =3, refit = True)

In [26]:
grid_tree.fit(X_train, y_train)

scores = pd.DataFrame(grid_tree.cv_results_)
scores[['params','mean_test_score','rank_test_score','split0_test_score','split1_test_score','split2_test_score']]

#split1_test_score , split0_test_score  = CV가 3인 경우 각각의 세트에서 테스트한 성능 수치

,params,mean_test_score,rank_test_score,split0_test_score,split1_test_score,split2_test_score
0,"{'max_depth': 1, 'min_samples_split': 2}",0.700000,5,0.700,0.7,0.70
1,"{'max_depth': 1, 'min_samples_split': 3}",0.700000,5,0.700,0.7,0.70
2,"{'max_depth': 2, 'min_samples_split': 2}",0.958333,3,0.925,1.0,0.95
3,"{'max_depth': 2, 'min_samples_split': 3}",0.958333,3,0.925,1.0,0.95
4,"{'max_depth': 3, 'min_samples_split': 2}",0.975000,1,0.975,1.0,0.95
5,"{'max_depth': 3, 'min_samples_split': 3}",0.975000,1,0.975,1.0,0.95


In [28]:
print(grid_tree.best_params_, grid_tree.best_score_)

{'max_depth': 3, 'min_samples_split': 2} 0.975


In [29]:
estimator = grid_tree.best_estimator_

#GridSearchCV의 best_estimator_는 이미 최적 학습이 됐으므로 별도 학습이 필요 없음
pred = estimator.predict(X_test)
print(accuracy_score(y_test,pred))

0.9666666666666667


일반적으로 GridSearchCV를 이용해 최적 하이퍼 파라미터 튜닝을 수행하고 test set에서 평가하는게 일반적인 ML 적용 방법

## 2.5 데이터 전처리

1) label encoding
- 숫자가 순서나 중요도로 인식돼서는 안된다.
- 그래서 선형 회귀와 같은 알고리즘에는 적용하면 안됨. --> 트리 계열은 okay

In [31]:
from sklearn.preprocessing import LabelEncoder

items = ['TV','냉장고','전자레인지','컴퓨터','선풍기','믹서','믹서']

encoder = LabelEncoder()
encoder.fit(items)
labels = encoder.transform(items)
print('encoding changed values : ', labels)
print('encoding class : ',encoder.classes_)

encoding changed values :  [0 1 4 5 3 2 2]
encoding class :  ['TV' '냉장고' '믹서' '선풍기' '전자레인지' '컴퓨터']


2) One-hot Encoding (Label encoding의 단점 보완)
- 주의1 : OneHoEncoder로 변환하기 전에 LabelEncoder로 모든 값이 숫자형으로 변환되어야 함.
- 주의2 : 입력 값으로 2차원 데이터가 필요하다.

In [34]:
from sklearn.preprocessing import OneHotEncoder

#LabelEncoder로 숫자형으로 변환
encoder = LabelEncoder()
encoder.fit(items)

#2차원으로 변환
labels = labels.reshape(-1,1)

#원-핫 인코딩 적용
ohencoder = OneHotEncoder()
ohencoder.fit(labels)
ohlabels = ohencoder.transform(labels)
print('원-핫 인코딩 데이터 \n', ohlabels.toarray())
print('원-핫 인코딩 데이터 차원 : ', ohlabels.shape)

원-핫 인코딩 데이터 
 [[1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]]
원-핫 인코딩 데이터 차원 (7, 6)


In [35]:
# 더 쉽게

df = pd.DataFrame({'item' : ['TV','냉장고','전자레인지','컴퓨터','선풍기','믹서','믹서']})
pd.get_dummies(df)

,item_TV,item_냉장고,item_믹서,item_선풍기,item_전자레인지,item_컴퓨터
0,1,0,0,0,0,0
1,0,1,0,0,0,0
2,0,0,0,0,1,0
3,0,0,0,0,0,1
4,0,0,0,1,0,0
5,0,0,1,0,0,0
6,0,0,1,0,0,0


3) 피처스케일링과 정규화

★ 회귀, 로지스틱 회귀, SVM --> 가우시안 분포(~N(0,1))로 가정을 해서 사전에 표준화를 하면 예측 성능 향상 good

In [37]:
iris = load_iris()
iris_data = iris.data
iris_df = pd.DataFrame(iris_data, columns = iris.feature_names)

print('feature의 평균 \n', iris_df.mean(),'\n\n')
print('feature의 분산 \n', iris_df.var())

feature의 평균 
 sepal length (cm)    5.843333
sepal width (cm)     3.057333
petal length (cm)    3.758000
petal width (cm)     1.199333
dtype: float64 


feature의 분산 
 sepal length (cm)    0.685694
sepal width (cm)     0.189979
petal length (cm)    3.116278
petal width (cm)     0.581006
dtype: float64


In [41]:
from sklearn.preprocessing import StandardScaler

scaler= StandardScaler()

scaler.fit(iris_df)
iris_scaled = scaler.transform(iris_df)

iris_df_scaled = pd.DataFrame(iris_scaled, columns = iris.feature_names)
print(iris_df_scaled.mean(),'\n\n')
print(iris_df_scaled.var())

## 모든 칼럼의 평균이 0에 근사하고 분산이 1에 근사하므로 scale 된 것

sepal length (cm)   -1.690315e-15
sepal width (cm)    -1.842970e-15
petal length (cm)   -1.698641e-15
petal width (cm)    -1.409243e-15
dtype: float64 


sepal length (cm)    1.006711
sepal width (cm)     1.006711
petal length (cm)    1.006711
petal width (cm)     1.006711
dtype: float64


In [43]:
#데이터가 가우시안 분포가 아닐 경우에 MinMaxScaler 이용
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

scaler.fit(iris_df)
iris_scaled = scaler.transform(iris_df)

iris_df_scaled = pd.DataFrame(iris_scaled, columns = iris.feature_names)
print(iris_df_scaled.min(),'\n\n')
print(iris_df_scaled.max())

# 모든 피처에 0~1 사이의 값으로 변환되는 스케일링 적용됨

sepal length (cm)    0.0
sepal width (cm)     0.0
petal length (cm)    0.0
petal width (cm)     0.0
dtype: float64 


sepal length (cm)    1.0
sepal width (cm)     1.0
petal length (cm)    1.0
petal width (cm)     1.0
dtype: float64


★ train data --> fit(), transform() 시켰으면 test data --> transform()만 실행해야함 
(다시 fit() 하면 train 시킨 정보가 날아가고 test에 맞춰서 다시 스케일링 시키기 때문)

그래서 가능하다면 전체 데이터의 스케일링 변환을 적용한 뒤 train / test data set split 하기